# ImmoWeb Data Cleaning
## Imports

In [42]:
import pandas as pd

## Create dataframe with cleaned datset

In [43]:
df = pd.read_csv('Hussain - Cleaned_Data.csv')
print(df.dtypes)

Id                      int64
Locality               object
Type                   object
SubType                object
Sale_Type              object
Price                   int64
Bedrooms                int64
Living_Area             int64
Is_Equiped_Kitchen       bool
Is_Furnished             bool
Is_Open_Fire             bool
Terrace                  bool
Terrace_Area            int64
Garden                   bool
Garden_Area             int64
Land_Surface            int64
Surface of the plot     int64
State                  object
Facades                 int64
Swim_pool                bool
dtype: object


## Fix datatypes

In [44]:
df['Locality'] = df['Locality'].astype('category')
df['Type'] = df['Type'].astype('category')
df['SubType'] = df['SubType'].astype('category')
df['State'] = df['State'].astype('category')
print(df.dtypes)

Id                        int64
Locality               category
Type                   category
SubType                category
Sale_Type                object
Price                     int64
Bedrooms                  int64
Living_Area               int64
Is_Equiped_Kitchen         bool
Is_Furnished               bool
Is_Open_Fire               bool
Terrace                    bool
Terrace_Area              int64
Garden                     bool
Garden_Area               int64
Land_Surface              int64
Surface of the plot       int64
State                  category
Facades                   int64
Swim_pool                  bool
dtype: object


## Drop duplicates

In [45]:
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}") #92 rows
df = df.drop_duplicates()
df = df.reset_index(drop=True)

Number of duplicate rows: 92


## Drop Columns/Rows

In [46]:
# Dropping Sale_Type because all are "Private"
df = df.drop(columns=['Sale_Type'])

In [47]:
# Drop all rows where Type = Other - we are only interested in Houses and Apartments, not garages/offices/covered parking etc.
count_other = df[df['Type'] == 'Other'].shape[0]
print(f"Number of rows with Type 'Other': {count_other}")
df = df[df['Type'] != 'Other']

Number of rows with Type 'Other': 2314


## Deal with 0 values in Living_Area

In [48]:
# Checking proportion of rows where Living_Area = 0
num_zero_area = df[df['Living_Area'] == 0].shape[0]
total_rows = df.shape[0]
proportion = num_zero_area / total_rows
print(f"Proportion of rows with Living_Area = 0: {proportion:.2%}")

# Impute with median
median_living_area = df[df['Living_Area'] > 0]['Living_Area'].median()
df['Living_Area'] = df['Living_Area'].replace(0, median_living_area)

df['Living_Area'] = df.groupby('Type', observed=True)['Living_Area'].transform(
    lambda x: x.replace(0, x.median())
)

print(df['Living_Area'].describe())


Proportion of rows with Living_Area = 0: 4.03%
count    11301.000000
mean       155.784355
std        121.705883
min         15.000000
25%         91.000000
50%        127.000000
75%        180.000000
max       3255.000000
Name: Living_Area, dtype: float64


## Deal with 0 values in Surface of the plot

In [49]:
# Checking proportion of rows where Living_Area = 0
num_zero_plot = df[df['Surface of the plot'] == 0].shape[0]
total_rows = df.shape[0]
proportion = num_zero_plot / total_rows
print(f"Proportion of rows where Surface of the plot = 0: {proportion:.2%}")

num_zero_land = df[df['Land_Surface'] == 0].shape[0]
total_rows = df.shape[0]
proportion = num_zero_land / total_rows
print(f"Proportion of rows where Land_Surface = 0: {proportion:.2%}")


Proportion of rows where Surface of the plot = 0: 53.88%
Proportion of rows where Land_Surface = 0: 2.86%


Seems like there was an initial mistake made during the scraping process where Land_Surface and Surface of the plot got switched.
Land_Surface has less missing values and the values are also larger than what can be seen in Surface of the plot which isn't logical.

Going to dicount Surface of the plot from this point as too much unreliable data.

Will use Land_Surface as variable for total surface (land + building)

In [ ]:
df = df.drop(columns=['Surface of the plot'])

# Check that grouping by type is a good strategy
# Yes, because Apartment and House have signifiantly different median sizes.
print(df.groupby('Type')['Land_Surface'].median())
# Remove unused categories
df['Type'] = df['Type'].cat.remove_unused_categories()



# Impute Land_Surface with median
median_land_surface = df[df['Land_Surface'] > 0]['Land_Surface'].median()
df['Land_Surface'] = df['Land_Surface'].replace(0, median_land_surface)

df['Land_Surface'] = df.groupby('Type', observed=True)['Land_Surface'].transform(
    lambda x: x.replace(0, x.median())
)

print(df['Land_Surface'].describe())


Type
Apartment    103.0
House        216.0
Other          NaN
Name: Land_Surface, dtype: float64
count    11301.000000
mean       281.328555
std        908.991313
min          1.000000
25%        100.000000
50%        146.000000
75%        234.000000
max      50760.000000
Name: Land_Surface, dtype: float64


C:\Users\T570\AppData\Local\Temp\ipykernel_8620\3584237065.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Type')['Land_Surface'].median())


Significant leap from 75% to max value - indicates outlier.
A min value of 1 is also weird.

## Change Booleans to Binary

In [51]:
df['Is_Equiped_Kitchen'] = df['Is_Equiped_Kitchen'].astype(int)
df['Is_Furnished'] = df['Is_Furnished'].astype(int)
df['Is_Open_Fire'] = df['Is_Open_Fire'].astype(int)
df['Terrace'] = df['Terrace'].astype(int)
df['Garden'] = df['Garden'].astype(int)
df['Swim_pool'] = df['Swim_pool'].astype(int)

print(df['Swim_pool'].unique())

[0 1]


## Save to new csv

In [52]:
df.to_csv('immo_cleaned_data.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'immo_cleaned_data.csv'